In [1]:
from netgen.occ import *
from ngsolve import Mesh

import sys
sys.path.append("../build/rigid_body")
from rigid_body import *
import rigid_body.bla as bla

import pythreejs as p3

from IPython.display import display
import ipywidgets as widgets

In [2]:
# set up OCC CAD model
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.5))
# important: move the center of mass into the origin
box = box.Move((-box.center[0], -box.center[1], -box.center[2]))

In [3]:
# from ngsolve.webgui import Draw
# Draw(mesh)

In [4]:
def mass_matrix_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"
    
    # copy the inertia matrix from netgen
    inertia_matrix = bla.Matrix(3,3)
    for i in range(3):
        for j in range(3):
            inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    center_of_mass = bla.Vector(3)
    for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    mass_matrix = mass_matrix_from_inertia(inertia_matrix, center_of_mass, obj.mass)
    return mass_matrix

mass_matrix = mass_matrix_from_solid(box)
# print(mass_matrix)
#for i in range(18):
#    for j in range(18):
#        print(F"{round(mass_matrix[i, j], 5)}, ", end="")
#    print("\\")


In [5]:
def extract_vertices(mesh: Mesh):
    "extracts a p3js compatible vertex list from a netgen.occ Mesh"
    combined_vertices = [] # flat array of vertices of all faces
    all_vertices = [vert.point for vert in mesh.vertices] # all available vertices

    # throw all vertices of all faces into combined_vertices
    for face in mesh.faces:
        for vertex in face.vertices:
            point = all_vertices[vertex.nr]
            combined_vertices.append(point)
            
    return combined_vertices

In [6]:
rbs = RBSystem()

In [7]:
# set up physics simulation object
r1 = RigidBody()
r1.setMass(mass_matrix)

p1 = Transformation()
p1.setTranslation(0,0,0)
p1.setRotation(0,0,1)
p1.setRotation(1,1,1)
p1.setRotation(2,2,1)
r1.q=p1

t1 = Transformation()
t1.setTranslation(0.000,0.1,0.1)
t1.setRotation(0,2,0.001)
t1.setRotation(2,0,-0.001)
r1.dq=t1



r2 = RigidBody()
r2.setMass(mass_matrix)

p2 = Transformation()
p2.setTranslation(1,1,1)
p2.setRotation(0,0,1)
p2.setRotation(1,1,1)
p2.setRotation(2,2,1)
r2.q=p2

t2 = Transformation()
t2.setTranslation(0.000,0.1,0)
t2.setRotation(0,2,0.02)
t2.setRotation(2,0,-0.02)
r2.dq=t2

#Add body to system
c1 = rbs.addBody(r1)
c1.pos=(0,0,0)
c2 = rbs.addBody(r2)
c2.pos=(0,0,0)
#b = Beam(c1,c2,1.732);
#rbs.addBeam(b)

# save a restore point for the reset button
rbs.saveState()


In [8]:
meshes = [mesh,mesh]

In [9]:
print(rbs.bodies()[0].q)

 Translation: 	0.000000 ,0.000000, , 0.000000
 Rotation: 	1.000000 ,0.000000, , 0.000000
 		0.000000 ,1.000000, , 0.000000
		0.000000 ,0.000000, , 1.000000



In [10]:
view_width = 600
view_height = 400
buffergeos = []
p3meshes = []
# set up pythreejs 3d object
for m in meshes:
    original_vertices = extract_vertices(m)
    buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(original_vertices, normalized=False)})
    buffergeom.exec_three_obj_method("computeVertexNormals") # gives normals for shading
    material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
    p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))
    buffergeos.append(buffergeom)
    p3meshes.append(p3mesh)

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")

# set up scene
scene = p3.Scene(children=[camera, key_light, ambient_light, grid]+p3meshes)
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [11]:
print(r2.dq)

 Translation: 	0.000000 ,0.100000, , 0.000000
 Rotation: 	0.000000 ,0.000000, , 0.020000
 		0.000000 ,0.000000, , 0.000000
		-0.020000 ,0.000000, , 0.000000



In [12]:
# play/reset button
play = widgets.Play(
    value=10,
    min=0,
    max=10000,
    step=1,
    interval=100/30,
    description="Press play",
    disabled=False
)

In [13]:
c2.pos

(0.0, 0.0, 0.0)

In [14]:
for m in p3meshes:
    m.matrixAutoUpdate = False # make mesh movable

def refreshMeshes():
    for i in range(0,len(meshes)):
        p3meshes[i].matrix=rbs.bodies()[i].q.asTuple()

def update():
    "update function, gets called every timestep; quasi main loop"
    rbs.simulate(1,10)
    refreshMeshes()

def observer(state):
    "event handler for clickable buttons"
    # if there is a change in time
    if state["name"] == "value":
        # it might be a reset
        if str(state["new"]) == "0":
            rbs.reset()
            refreshMeshes()
        # or it might be a progress in time
        else:
            update()
    # repeat is used as an alias to reset
    elif state["name"] == "repeat":
        rbs.reset()
        refreshMeshes()

play.observe(observer)


In [15]:
display(widgets.HBox([play, widgets.HTML("<b>click-and-drag to rotate, scroll to zoom, right-click-and-drag to move<b>")]))
display(renderer)
refreshMeshes()

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…